## 🛠️ **ETL (Extract, Transform, Load)**
### **📂Procesamos el 2do archivo: `user_items.json.gz`**


📂 **user_reviews.json**:: 
- 🗃️ `Items `**:** Desanidado: era una lista de diccionarios
- 🔄 `Playtime_forever `: Transformamos los minutos a horas

#### 📦 **EXTRACT**

#### **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
from data_utils import data_type_check, duplicados_columna
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

#### **Descomprimimos el archivo gz** 

In [2]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/users_items.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)


Archivo descomprimido: ../0 Dataset/users_items.json


### **📂Procesamos el 2do archivo: `user_items.json.gz`**


- Tomamos los datos del archivo JSON, transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [3]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("../0 Dataset/users_items.json", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_users_items = pd.DataFrame(rows)
#Veamos unos registros al asar
df_users_items.sample(2)

,user_id,items_count,steam_id,user_url,items
74109,tooditoo,3,76561198081199422,http://steamcommunity.com/id/tooditoo,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
2495,chocolatemanj,20,76561198067336542,http://steamcommunity.com/id/chocolatemanj,"[{'item_id': '240', 'item_name': 'Counter-Stri..."


Usando la funcion personalizada `data_type_check` invocada desde `data_utils.py` podemos observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


In [4]:
data_type_check(df_users_items)


 Resumen del dataframe 'df_users_items': 

Dimensiones:  (88310, 5)
       columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0      user_id       100.0      0.0            0    object
1  items_count       100.0      0.0            0     int64
2     steam_id       100.0      0.0            0    object
3     user_url       100.0      0.0            0    object
4        items       100.0      0.0            0    object


Este conjunto contiene 5 columnas y 88310 filas sin nulos.

In [5]:
df_users_items.sample(1)

,user_id,items_count,steam_id,user_url,items
85811,76561198106185045,1,76561198106185045,http://steamcommunity.com/profiles/76561198106...,"[{'item_id': '205790', 'item_name': 'Dota 2 Te..."


### 🔁 **TRANSFORM**

#### Columna **'items'** (es una lista de diccionarios)
Desanidado de items, que contiene:
- Item_name
- playtime_forever
- playtime_2weeks


In [6]:
#Creamos una nueva fila para cada elemento de la lista en la columna items
df_users_items = df_users_items.explode("items").reset_index()
#Eliminamos la columna index
df_users_items = df_users_items.drop(columns="index")
# Creamos una nueva columna para cada elemento de la lista en la columna items
df_users_items = pd.concat([df_users_items, pd.json_normalize(df_users_items['items'])], axis=1)
#Una vez extraidos eliminamos la columna items
df_users_items.drop(columns=['items'], inplace=True)

data_type_check(df_users_items)


 Resumen del dataframe 'df_users_items': 

Dimensiones:  (5170015, 8)
            columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0           user_id      100.00     0.00            0    object
1       items_count      100.00     0.00            0     int64
2          steam_id      100.00     0.00            0    object
3          user_url      100.00     0.00            0    object
4           item_id       99.67     0.33        16806    object
5         item_name       99.67     0.33        16806    object
6  playtime_forever       99.67     0.33        16806   float64
7   playtime_2weeks       99.67     0.33        16806   float64


#### Playtime_forever: Transformamos los minutos a horas 

In [7]:
# mostramos un playtime_forever de ejemplo
print("Antes de limpiar")
df_users_items['playtime_forever']

Antes de limpiar


0          6.0
1          0.0
2          7.0
3          0.0
4          0.0
          ... 
5170010    0.0
5170011    3.0
5170012    4.0
5170013    3.0
5170014    NaN
Name: playtime_forever, Length: 5170015, dtype: float64

In [8]:
#borramos donde no haya tiempo de juego
df_users_items = df_users_items[df_users_items['playtime_forever'] != 0]
#Convertimos los minutos a horas
df_users_items['playtime_forever'] = df_users_items['playtime_forever'] / 60

df_users_items['playtime_forever'].fillna(0, inplace=True)

# mostramos un playtime_forever de ejemplo
print("Despues de limpiar")
df_users_items['playtime_forever']


Despues de limpiar


0           0.100000
2           0.116667
8          78.883333
9          30.883333
10          5.550000
             ...    
5170008     0.716667
5170011     0.050000
5170012     0.066667
5170013     0.050000
5170014     0.000000
Name: playtime_forever, Length: 3302052, dtype: float64

Como ya tenemos la informacion de tiempo de juego acumulado por usuario, borramos esta columna innecesaria


In [9]:
df_users_items = df_users_items.drop('playtime_2weeks', axis=1)

#### item_name

In [10]:
df_users_items['item_name'].fillna(0, inplace=True)
df_users_items['item_name'] = df_users_items['item_name'].astype(str)

#### steam_id y user_url
- Eliminamos las columnas irrelevantes que no nos riven

In [11]:
df_users_items.drop(["user_url","steam_id"], axis=1, inplace=True)

#### user_id

In [12]:

# Eliminar espacios en blanco al principio y al final de los valores
df_users_items['user_id'] = df_users_items['user_id'].str.strip()
# Llenar valores nulos con un valor específico (por ejemplo, cadena vacía)
df_users_items['user_id'].fillna(' ', inplace=True)
# Eliminar caracteres especiales o no imprimibles
df_users_items['user_id'] = df_users_items['user_id'].str.replace(r'\W', '')
# Convertir a tipo string
df_users_items = df_users_items.astype({'user_id': 'string'})


#### **item_id**

In [13]:
df_users_items['item_id'].fillna(0, inplace=True)
df_users_items = df_users_items.drop_duplicates(keep='first')
df_users_items['item_id'] = df_users_items['item_id'].astype(int)

data_type_check(df_users_items)



 Resumen del dataframe 'df_users_items': 

Dimensiones:  (3263076, 5)
            columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0           user_id       100.0      0.0            0  string[python]
1       items_count       100.0      0.0            0           int64
2           item_id       100.0      0.0            0           int32
3         item_name       100.0      0.0            0          object
4  playtime_forever       100.0      0.0            0         float64


### 📤 **LOAD**

In [14]:
# Define la ruta del archivo Parquet
ruta_parquet = os.path.abspath('../0 Dataset/1.2_user_items_LISTO.parquet')

# Convertir el DataFrame de pandas a una tabla de PyArrow
df_users_items = pa.Table.from_pandas(df_users_items)
# Escribir la tabla en un archivo Parquet
pq.write_table(df_users_items, ruta_parquet)

Eliminamos el archivo descomprimidos que ahora tenemos limpio y liviano en formato parquet

In [15]:
# Lista de archivos descomprimidos
gz_descomprimidos = [
    '../0 Dataset/users_items.json'
]

# Eliminar archivos descomprimidos
for archivo_json in gz_descomprimidos:
    try:
        os.remove(archivo_json)
        print(f'Archivo eliminado: {archivo_json}')
    except FileNotFoundError:
        print(f'Archivo no encontrado: {archivo_json}')

Archivo eliminado: ../0 Dataset/users_items.json


Observamos el tiempo de ejecucion total de nuestro proceso ETL 🔥

In [16]:
# Obtener el tiempo de finalización
end_time = time.time()
# Calcular el tiempo total de ejecución
total_time = end_time - start_time
# Convertir a minutos y redondear a 2 decimales
total_time_minutes = round(total_time / 60, 2)
# Imprimir resultados
print(f"Tiempo total de ejecución de este ipynb: {total_time_minutes} minutos")

Tiempo total de ejecución de este ipynb: 3.42 minutos
